# Statistical Analysis

In [1]:
import numpy as np
from scipy.stats import poisson as pois
from scipy.stats import norm
import matplotlib.pyplot as plt

import nbimporter
import prepData as prep
import fitFunc as fits

import multiprocess as mp

## Significance

Assuming the presence of a signal, we want to compute its significance on the observed dataset.  

The first steps are rescaling the dataset and fitting the background and signal functions.  
We then compute the likelihood ratio of the observed data $q_0^{obs}$ between the signal and null hypothesis:

$$q_0^{obs} = -2 \cdot \log \left( \cfrac{\mathcal{L} \left(Data | 0, \hat{\theta}_0 \right)}{\mathcal{L} \left(Data | \hat{\mu}, \hat{\theta}_\hat{\mu} \right)}\right)$$

The ^ symbol indicates the values that optimize the fits.

The value of $q_0^{obs}$ has no meaning by itself, so we generate n = 10,000 toy datasets from the expected values given by the background fit and repeat the analysis for every new dataset. Both for the likelihood ratio and for the toy dataset generation, a normal approximation has been used instead of the formal Poisson distribution. 

The original $q_0^{obs}$ is thus compared with the distribution of $q_0$ obtained from the toy datasets, and the p-value is computed:

$$p_0 = P \left( q_0 \ge q_0^{obs} \right) = \int_{q_0^{obs}}^{+\infty} f(q_0 | 0, \hat{\theta}_0) \,dx $$ 

The significance is expressed as the number of $\sigma$s needed to achieve an equivalent p-value in a standard normal distribution:

$$z = \Phi^{-1} \left(1 - p_0 \right)$$  

This process is repeted using every possible frequency as $x_0$, the center of the signal function.

## Confidence Interval

The goal of this analysis is to set a limit on the strength of the signal. In order to do so, we scan over certain values of $\mu$ and do the following analysis.

In addition to fitting the background, we fit the signal twice: one time we let the $\mu$ run free to find $\hat{\mu}$, while the other we keep it fixed to a certain value $\mu$.  
The likelihood ratio is then computed as:

$$q^{obs} \left( \mu \right) = -2 \cdot \log \left( \cfrac{\mathcal{L} \left(Data | \mu, \hat{\theta}_{\mu} \right)}{\mathcal{L} \left(Data | \hat{\mu}, \hat{\theta}_\hat{\mu} \right)}\right)$$

We then generate two sets of n = 10,000 toy datasets each, one as before from the expected values given by the background fit while the other from the signal fit with fixed $\mu$. We compare the original $q^{obs}(\mu)$ with the distribution of $q(\mu)$ from the toy datasets and compute the two probabilities:

$$    p_{\mu} = P \left(q(\mu) \ge q^{obs}(\mu) | \mu s + b \right)$$
$$1 - p_{b}   = P \left(q(\mu) \ge q^{obs}(\mu) | b \right)$$

and take their ratio.  

$$CL_S = \cfrac{p_{\mu}}{1 - p_{b}}$$

Finally, we take as the 95% confidence interval limit the value of $\mu$ so that the ratio is equal to 0.05:

$$\mu^{95 \%CL} = \mu \; \big| \;CL_S = 0.05$$

## Code Implementation

### Toy datasets

In [2]:
def gen_toydataset(values, n, N=1365500):
    toy_dataset = norm.rvs(loc=values, scale=prep.calc_weights(values, N), size=(n, len(values)))

    return toy_dataset

In [3]:
def multiple_toydataset(InfoDataset, n, data_type="fft"):
    res_list = []
    for i in range(n):
        run_list = []
        for i_run, run in enumerate(InfoDataset):
            toy_data = gen_toydataset(run[data_type], 1, run["N"]).flatten()
            toy_dict = {"name":run["name"],
                        "N":run["N"],
                        "center":run["center"],
                        "ref":run["ref"],
                        "freq":run["freq"],
                        "fft":toy_data,
                        "weights":prep.calc_weights(toy_data, run["N"])}
            run_list.append(toy_dict)
        res_list.append(run_list)
        
    return res_list

### Likelihood ratio

In [4]:
# compute likelihood ratios of two hypotheses
def lh_ratio(y, null_vals, alt_vals, null_weights=None, alt_weights=None, N=1365500):
    
    if null_weights is None:
        null_weights = prep.calc_weights(null_vals, N)
    if alt_weights is None:
        alt_weights = prep.calc_weights(alt_vals, N)
    
    # compute log likelihoods
    LogLike_null = sum(norm.logpdf(x=y, loc=null_vals, scale=null_weights))
    LogLike_alt  = sum(norm.logpdf(x=y, loc=alt_vals,  scale=alt_weights))
    
    #LogLike_null = sum(pois.logpmf(fft.astype(int), model_null.best_fit.astype(int)))
    #LogLike_alt  = sum(pois.logpmf(fft.astype(int), model_alt.best_fit.astype(int)))
    
    # ratio
    q = -2 * (LogLike_null - LogLike_alt)
    
    return q

In [5]:
def multiple_lh_ratio(data_list, name1, null_list, name2, alt_list, name3):
    # get dataset to compute likelihood ratio on from each list of results
    y = np.zeros(shape=(len(data_list), len(data_list[0][name1])))
    null         = np.zeros_like(y)
    null_weights = np.zeros_like(y)
    alt          = np.zeros_like(y)
    alt_weights  = np.zeros_like(y)
    for i_run, run in enumerate(data_list):
        y[i_run] = run[name1]
        null[i_run] = null_list[i_run][name2]
        null_weights[i_run] = null_list[i_run]["weights"]
        alt[i_run] = alt_list[i_run][name3]
        alt_weights[i_run] = alt_list[i_run]["weights"]
        
    q = lh_ratio(y.flatten(), null.flatten(), alt.flatten(),
                 null_weights.flatten(), alt_weights.flatten())
    
    return q

In [6]:
def multiple_calc_qmu(toyData, x0,mu_fix):
    
    n_toy = len(toyData)
    
    # compute likelihood ratio for toy dataset
    q_mu = np.empty(n_toy)
    for i_toy,toy in enumerate(toyData):
        
        toy_bkg_params = fits.multipleFitBKG(toy)
        
        fix_toy=fits.multipleFitSIG(toy, toy_bkg_params, x_0=x0,mu_init=mu_fix,mu_vary=False)
        fitSig_toy=fits.multipleFitSIG(toy, toy_bkg_params, x_0=x0)
        
        q_mu[i_toy] = multiple_lh_ratio(toy, "fft",fix_toy, "sig_bestFit",fitSig_toy,"sig_bestFit")
        
    return q_mu

In [7]:
def p_value(q_obs, q):
    p0 = sum(q >= q_obs)/len(q)
    return p0

### Complete test

In [8]:
def stat_test(InfoDataset,x_0,mu_fix=1.0,nToy=10000,calc_CI=False,calc_z=False,verbose=False,draw=False):
    
    #fit background once
    fitBkg = fits.multipleFitBKG(InfoDataset)
    
    # generate toy datasets from bkg and fit them
    toy_0 = multiple_toydataset(fitBkg, n=nToy, data_type="bkg_bestFit")
    
    z     = np.empty(len(x_0))
    mu_CI = np.empty(len(x_0))
    r = np.empty(len(x_0))
    
    for i_x0,x0 in enumerate(x_0):
        
        fitSig = fits.multipleFitSIG(InfoDataset, fitBkg, x_0=x0)
        
        # compute significance
        if calc_z:
            z[i_x0] = multipleSignificance(InfoDataset, fitBkg, fitSig, toy_0, x0, draw)
        
        if calc_CI:
            if i_x0 == 0:
                mu_CI[i_x0] = multipleCI(InfoDataset,fitBkg,fitSig,toy_0,x0,mu_fix,verbose,draw)
        
            else:
                mu_CI[i_x0] = multipleCI(InfoDataset,fitBkg,fitSig,toy_0,x0,mu_CI[i_x0-1],verbose,draw)
        
            if verbose:
                print("Testing x0:",x0)
                print("mu_CI:",mu_CI[i_x0],"\tr:",r[i_x0])
                print("---------------------")
    
    return z,mu_CI    

In [9]:
def multipleSignificance(InfoDataset, fitBkg, fitSig, toy_0, x_0, draw=False):
    
    # compute likelihood ratio of observed data
    q0_obs = multiple_lh_ratio(InfoDataset, "fft",
                               fitBkg, "bkg_bestFit",
                               fitSig, "sig_bestFit")     
    
    n_toy = len(toy_0)
    
    # analyze toy datasets
    q0 = np.empty(n_toy)
    for i_toy in range(n_toy):
        # fit background over toy dataset
        toy_fitBkg = fits.multipleFitBKG(toy_0[i_toy])
        # fit signal
        toy_fitSig = fits.multipleFitSIG(toy_0[i_toy], toy_fitBkg, x_0)
      
        # compue likelihood ratio of toy dataset 
        q0[i_toy] =  multiple_lh_ratio(toy_0[i_toy], "fft",
                                       toy_fitBkg, "bkg_bestFit",
                                       toy_fitSig, "sig_bestFit")
    
    # plot significance distribution
    if(draw):
        plot_lhratio(q0_obs, q0, x_0=x_0)
        
    # compute significance
    p0 = p_value(q0_obs, q0)
    z = norm.ppf(1-p0)
        
    return z

In [10]:
def multipleCI(InfoDataset, fitBkg, fitSig,toy_0,x_0, mu_fix,verbose=False,draw=False):
    
    n_toy = len(toy_0)
    
    # scan for mu
    q_mu_obs_prev = 0            # save distribution and parameters
    q_mu_prev = np.empty(n_toy)  # to plot optimal result
    q0_prev = np.empty(n_toy)
    
    r_prev = 1e10
    mu_95 = 0
    
    sign_prev = 0
        
    mu_test=mu_fix
    cross_check=0
    
    all_mu=[]
    while True:
                
        if mu_test in all_mu:
            break
        
        fix = fits.multipleFitSIG(InfoDataset, fitBkg, x_0=x_0,mu_init=mu_test,mu_vary=False)
        
        # compute likelihood ratio of observed data
        q_mu_obs = multiple_lh_ratio(InfoDataset, "fft",
                                       fix, "sig_bestFit",
                                       fitSig, "sig_bestFit")
        
        if(verbose):
            print("Mu: ", mu_test, "   q(mu)_obs =", q_mu_obs)
        
        # generate toy datasets from fixed mu
        toy_fix = multiple_toydataset(fix, n=n_toy, data_type="sig_bestFit")
        
        q_mu=multiple_calc_qmu(toy_fix,x_0,mu_test)
        q0=multiple_calc_qmu(toy_0,x_0,mu_test)
        
        # compute p-values
        p_mu = p_value(q_mu_obs, q_mu)
        p_b  = p_value(q_mu_obs, q0)
        
        # compute ratio
        r = p_mu/p_b
            
        if(verbose):
            print("p_mu =", p_mu, "  p_b =", p_b, "  ratio =", r)
            print(cross_check)
            print(all_mu, '\n')
            
        # check results to proceed with the mu scan:
        # if the ratio is close eneough to target we save the results and stop
        # else we check if we are under- or overshooting and correct the estimate
        # if we cross the target we stop and take the best result between
        # current and previous step
        
                
        #check if the next mu has been already tested
        all_mu.append(mu_test)
        
        if math.isnan(r) or math.isinf(r):
            mu_test = mu_test - 3
            if mu_test < 0: mu_test=1
            continue
            
        reached_target, crossed_target, is_current_worse = False, False, False
        
        reached_target = (np.abs(r - 0.05) <= 0.01)
        if not reached_target:
            sign = np.sign(r - 0.05)
            crossed_target = (sign*sign_prev == -1)
            
            if crossed_target:
                is_current_worse = (np.abs(r - 0.05) > np.abs(r_prev - 0.05))
                cross_check+=1
        
        # update best estimate for every case except the last
        if not is_current_worse:
            q_mu_obs_prev = q_mu_obs            
            q_mu_prev = q_mu  
            q0_prev = q0
            mu_95 = mu_test
            
        if reached_target or cross_check==2:
            cross_check=0
            break
        
        #adaptive step algorithm
        if r==0: r=0.14
        
        step=0   
        check=r-0.05
        
        if np.abs(check) > 0.5:
            step = sign*2
        elif ((np.abs(check) > 0.2) and (np.abs(check) < 0.5)):
            step = sign*7/5
        elif ((np.abs(check) > 0.08) and (np.abs(check) < 0.2)):
            step = sign
        else:
            step = sign*2/5
            
        # update mu if we did not exit the loop
        mu_test = mu_test + step*5
        r_prev = r
        sign_prev = sign

            
        if mu_test <= 0:
            mu_test=1
            
    # plot significance distribution
    if(draw):
        plot_lhratio(q_mu_obs_prev, q0_prev, q_mu_prev, x_0, mu_95)
            
    return(mu_95, r)

### Parallelization

In [11]:
def stat_test_parallel(runsData, x0, mu_init, njobs=10):
    
    x_0 = np.split(x0, njobs)
    manager = mp.Manager()
    results = manager.list()
    
    def worker(runsData, x_0, mu_init, results):
        result = stat_test(runsData, x_0, mu_init)
        results.append(result)
    
    processes = []
    for i in range(njobs):
        p = mp.Process(target=worker, args=(runsData, x_0[i], mu_init[i], results))
        processes.append(p)
        p.start()
    
    for p in processes:
        p.join()
        
    return list(results)

### Useful functions

In [12]:
def plot_lhratio(q_obs, q0, q_alt=np.array([]), x_0=0, mu_95=0):
    # prepare canvas
    fig = plt.figure(figsize=(8, 5))
    
    # plot q0 distribution
    #N = len(q0)
    #binning = int(np.sqrt(N))
    n, bins, _ = plt.hist(q0, bins="auto", density=True, alpha=0.5,
                          facecolor='lightblue', edgecolor='black', label='Toy Experiments: background')
    lineheight = max(n)
    if np.any(q_alt):
        n_alt, bins_alt, _ = plt.hist(q_alt, bins="auto", density=True, alpha=0.5,
                                      facecolor='salmon', edgecolor='black', label='Toy Experiments: signal')
        lineheight = max(lineheight, max(n_alt))
    
    plt.vlines(q_obs, 0, lineheight, colors='forestgreen', linestyles='--', label='Observed Data')
    
    plt.legend(loc='upper right')
    
    title = "X_0 = " + str(x_0)
    if np.any(q_alt):
        title = title + "    mu 95%CL = " + str(mu_95)
    
    plt.title(title)
    plt.xlabel('q')
    plt.ylabel('PDF')
    
    plt.show()